In [14]:
import duckduckgo_search as ddg # Make sure the function name is correctly spelled and capitalized
from fastcore.all import *

def search_images(term, max_images=200): return L(ddg.DDGS().images(term, max_results=max_images)).itemgot('image')

In [15]:
from fastdownload import download_url
from fastai.vision.all import *

In [16]:
path = Path('dataset')
from time import sleep
def collect_data(searches,size):
  for o in searches:
      dest = (path/o)
      dest.mkdir(exist_ok=True, parents=True)
      download_images(dest, urls=search_images(f'{o} photo'))
      resize_images(path/o, max_size=size, dest=path/o)

In [19]:
def check_data():
    failed = verify_images(get_image_files(path))
    failed.map(Path.unlink)

In [32]:
def set_centered(root,window_width,window_height):
    # get the screen dimension
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()

    # find the center point
    center_x = int(screen_width/2 - window_width / 2)
    center_y = int(screen_height/2 - window_height / 2)

    # set the position of the window to the center of the screen
    root.geometry(f'{window_width}x{window_height}+{center_x}+{center_y}')

In [128]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from PIL import Image, ImageTk

class ImageClassifierGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Classifier")
        
        self.welcome_label = tk.Label(root, text="Welcome to image classifier builder",bg="#48A099",padx=505,pady=5)
        self.welcome_label.pack()
        self.classes_label = tk.Label(root, text="just enter classes name and we will build the model for you",bg="#48A099",padx=505,pady=1)
        self.classes_label.pack()
        
        self.classes_label = tk.Label(root, text="Enter Classes Name (comma separated):",pady=5)
        self.classes_label.pack()

        self.classes_entry = tk.Entry(root)
        self.classes_entry.pack()

        # Submit Button
        self.submit_button = tk.Button(root, text="download dataset and create classifier for these classes", command=self.create_classifier)
        self.submit_button.pack()

        # Upload Image for Testing
        self.upload_button = tk.Button(root, text="Upload Image for Testing", command=self.upload_image)
        self.upload_button.pack()

        # Display Image
        self.image_label = tk.Label(root)
        self.image_label.pack()

        # Download Model Button
        self.download_button = tk.Button(root, text="Download Model", command=self.download_model)
        self.download_button.pack()
        
#         upload model
        self.upload_model_button = tk.Button(root, text="Upload Model", command=self.upload_model)
        self.upload_model_button.pack()
    def upload_model(self):
        print(path.parent)
        self.learn = load_learner(path.parent/'export.pkl')
        messagebox.showinfo("Info","model uploaded")
    
    def create_classifier(self):
        # This is where you'll add code to create the classifier
        class_names = self.classes_entry.get().split(',')
        collect_data(class_names,10)
        check_data()
        dls = DataBlock(
        blocks=(ImageBlock, CategoryBlock),
        get_items=get_image_files,
        splitter=RandomSplitter(valid_pct=0.2, seed=42),
        get_y=parent_label,
        item_tfms=Resize(192),

        batch_tfms=aug_transforms(mult=2)
        ).dataloaders(path)
        self.learn = vision_learner(dls, resnet18, metrics=error_rate)
        h = self.learn.fine_tune(3)
        self.train_label = tk.Label(root, text="Enter Classes Name (comma separated):")
        self.train_label.pack()
        messagebox.showinfo("Info", f"Classifier for  classes: {', '.join(class_names)} created!")

    def upload_image(self):
        # This is where you'll handle the uploaded image
        file_path = filedialog.askopenfilename()
        if file_path:
            img = Image.open(file_path)
            img.thumbnail((250, 250))
            uploaded_image = PILImage.create(img)
            img = ImageTk.PhotoImage(img)
            self.image_label.config(image=img)
            self.image_label.image = img
            
            # You can add code here to test the uploaded image with your classifier
            messagebox.showinfo("Info",f"prediction is {self.learn.predict(uploaded_image)}")
    def download_model(self):
        # This is where you'll handle model downloading
        self.learn.export()
        messagebox.showinfo("Info", "Model downloaded successfully!")

if __name__ == "__main__":
    root = tk.Tk()
    set_centered(root,500,400)
    app = ImageClassifierGUI(root)
    root.mainloop()


In [132]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk

class ImageClassifierGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Classifier")

        # Configure the root window background color
        self.root.configure(bg="#E0F7FA")

        # Welcome Label
        self.welcome_label = tk.Label(root, text="Welcome to Image Classifier Builder", bg="#48A099", fg="white", font=("Arial", 16, "bold"), padx=20, pady=10)
        self.welcome_label.pack(fill=tk.X)

        # Instruction Label
        self.instruction_label = tk.Label(root, text="Just enter class names and we will build the model for you", bg="#48A099", fg="white", font=("Arial", 12), padx=20, pady=5)
        self.instruction_label.pack(fill=tk.X)

        # Classes Label
        self.classes_label = tk.Label(root, text="Enter Class Names (comma separated):", bg="#E0F7FA", font=("Arial", 12), pady=10)
        self.classes_label.pack()

        # Classes Entry
        self.classes_entry = tk.Entry(root, font=("Arial", 12), width=50)
        self.classes_entry.pack(pady=(0, 15))

        # Submit Button
        self.submit_button = tk.Button(root, text="Download Dataset and Create Classifier", command=self.create_classifier, bg="#4CAF50", fg="white", font=("Arial", 12, "bold"), pady=5)
        self.submit_button.pack(pady=5)

        # Upload Image for Testing Button
        self.upload_button = tk.Button(root, text="Upload Image for Testing", command=self.upload_image, bg="#2196F3", fg="white", font=("Arial", 12, "bold"), pady=5)
        self.upload_button.pack(pady=5)

        # Image Display Label
        self.image_label = tk.Label(root, bg="#E0F7FA")
        self.image_label.pack(pady=10)

        # Download and Upload Model Buttons
        self.model_frame = tk.Frame(root, bg="#E0F7FA")
        self.model_frame.pack(pady=10)

        self.download_button = tk.Button(self.model_frame, text="Download Model", command=self.download_model, bg="#FF9800", fg="white", font=("Arial", 12, "bold"), padx=10, pady=5)
        self.download_button.pack(side="left", padx=5)

        self.upload_model_button = tk.Button(self.model_frame, text="Upload Model", command=self.upload_model, bg="#F44336", fg="white", font=("Arial", 12, "bold"), padx=10, pady=5)
        self.upload_model_button.pack(side="right", padx=5)

    def upload_model(self):
        self.learn = load_learner(path.parent/'export.pkl')
        messagebox.showinfo("Info", "Model uploaded successfully!")

    def create_classifier(self):
        # This is where you'll add code to create the classifier
        class_names = self.classes_entry.get().split(',')
        collect_data(class_names,10)
        check_data()
        dls = DataBlock(
        blocks=(ImageBlock, CategoryBlock),
        get_items=get_image_files,
        splitter=RandomSplitter(valid_pct=0.2, seed=42),
        get_y=parent_label,
        item_tfms=Resize(192),

        batch_tfms=aug_transforms(mult=2)
        ).dataloaders(path)
        self.learn = vision_learner(dls, resnet18, metrics=error_rate)
        h = self.learn.fine_tune(3)
        self.train_label = tk.Label(root, text="Enter Classes Name (comma separated):")
        self.train_label.pack()
        messagebox.showinfo("Info", f"Classifier for  classes: {', '.join(class_names)} created!")

    def upload_image(self):
        # This is where you'll handle the uploaded image
        file_path = filedialog.askopenfilename()
        if file_path:
            img = Image.open(file_path)
            img.thumbnail((250, 250))
            uploaded_image = PILImage.create(img)
            img = ImageTk.PhotoImage(img)
            self.image_label.config(image=img)
            self.image_label.image = img
            
            # You can add code here to test the uploaded image with your classifier
            messagebox.showinfo("Info",f"prediction is {self.learn.predict(uploaded_image)}")

    def download_model(self):
        self.learn.export()
        messagebox.showinfo("Info", "Model downloaded successfully!")

if __name__ == "__main__":
    root = tk.Tk()
    root.geometry("600x600")  # Set window size
    app = ImageClassifierGUI(root)
    root.mainloop()
